In [1]:
from typing import Tuple
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as sp
import sys

sns.set()



OUTPUT_TEMPLATE = (
    "Initial t-test p-value (invalid):\t\t{initial_ttest_p:.3g}\n"
    "Normality p-value of original data:\t\t{initial_weekday_normality_p:.3g}, {initial_weekend_normality_p:.3g}\n"
    "Equal variance p-value of original data:\t\t{initial_levene_p:.3g}\n"
    "Normality p-value of transformed data:\t\t{transformed_weekday_normality_p:.3g}, {transformed_weekend_normality_p:.3g}\n"
    "Equal variance p-value of transformed data:\t{transformed_levene_p:.3g}\n"
    "Normality p-value of weekly data:\t\t\t{weekly_weekday_normality_p:.3g}, {weekly_weekend_normality_p:.3g}\n"
    "Equal variance p-value of weekly data:\t\t{weekly_levene_p:.3g}\n"
    "Weekly t-test p-value:\t\t\t\t{weekly_ttest_p:.3g}\n"
    "Mann–Whitney U test p-value:\t\t\t{utest_p:.3g}"
)


def read_data(path: str) -> pd.DataFrame:
    # do not modify
    return pd.read_json(path, lines=True) 


def split_data(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # do not modify
    wd = df.query("is_weekend==False")
    we = df.query("is_weekend==True")
    return wd, we


def draw_histogram(df: pd.DataFrame, title: str = None) -> Figure:
    # do not modify
    fig, ax = plt.subplots(1, 1, dpi=100)
    ret = sns.histplot(data=df, x='comment_count', hue='is_weekend', ax=ax)
    if title:
        ret.set(title=title)
    return fig



In [10]:
path = "/Users/antonioslagarias/IFT6758/hw3-en/data/reddit-counts.json.gz"
#if len(sys.argv) > 1:
#    path = sys.argv[1]

# load data
raw_df = read_data(path)

In [11]:
raw_df

,date,subreddit,comment_count
0,2012-02-20,newfoundland,7
1,2015-01-26,Manitoba,1
2,2013-09-07,Yukon,2
3,2014-02-15,saskatchewan,5
4,2014-07-06,canada,1652
...,...,...,...
15465,2012-05-21,Quebec,365
15466,2012-05-21,britishcolumbia,4
15467,2013-09-07,britishcolumbia,5
15468,2011-09-10,Quebec,2


In [20]:
import datetime

raw_df["date"][0].year



AttributeError: 'Timestamp' object has no attribute 'dt'

In [ ]:
type(raw_df

In [81]:
# TODO - Complete this method
def process_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the raw DataFrame:

    1. Keep only the 'Canada' subreddit
    2. Keep only the years 2012 and 2013
    3. Add a new column 'is_weekend' with a Boolean value of True/False

    Args:
    df(pd.DataFrame): Dataframe to process; contains the columns
    'date', 'subreddit', 'comment_count' by default

    Returns:
    pd.DataFrame: Must have at least the columns: 'comment_count', 'date', 'is_weekend'
    """
    df = df.copy()  #copy so that you don't modify the original dataframe
    df = df[df['subreddit'] == "canada"]
    
    df = df[df['date'].dt.year.isin([2012, 2013])]
    df['is_weekend'] = df['date'].dt.weekday.isin([5,6])
    # TODO: Filtrez sur years, subreddit, et ajoutez une colonne boolean 'is_weekend' 
    return df

In [82]:
raw_df

,date,subreddit,comment_count
0,2012-02-20,newfoundland,7
1,2015-01-26,Manitoba,1
2,2013-09-07,Yukon,2
3,2014-02-15,saskatchewan,5
4,2014-07-06,canada,1652
...,...,...,...
15465,2012-05-21,Quebec,365
15466,2012-05-21,britishcolumbia,4
15467,2013-09-07,britishcolumbia,5
15468,2011-09-10,Quebec,2


In [83]:
other=process_data(raw_df)

In [35]:
wd=other[other["weekend"]==False]

In [36]:
we=other[other["weekend"]==True]

In [43]:
sp.ttest_ind(wd["comment_count"], we["comment_count"]).pvalue

np.float64(1.3005502847208094e-58)

In [42]:
sp.normaltest(wd["comment_count"])

NormaltestResult(statistic=np.float64(32.21804641032879), pvalue=np.float64(1.0091137251707994e-07))

In [44]:
sp.fligner(wd["comment_count"], we["comment_count"]).pvalue

np.float64(0.06604578026604031)

In [45]:
# TODO - Complétez cette méthode
def tests(wd: pd.DataFrame, we: pd.DataFrame, verbose: bool = False) -> Tuple[float, float, float, float]:
    """
    Performs a t-test between the two inputs, checking whether the mean of the two distributions is
    the same. It also checks whether the two input data sets have a normal distribution and the
    same variance (a requirement for the t-test).

    Reference: https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests

    Arguments:
    wd (pd.DataFrame): weekday data
    we (pd.DataFrame): weekend data
    verbose (bool): Whether to display the results

    Returns:
    Tuple[float, float, float, float]: p_test, p_wd_isnormal, p_we_isnormal, p_vartest
    """
    p_ttest, p_wd_normal, p_we_normal, p_vartest = None, None, None, None

    p_ttest=sp.ttest_ind(wd["comment_count"], we["comment_count"]).pvalue
    p_wd_normal=sp.normaltest(wd["comment_count"]).pvalue
    p_we_normal=sp.normaltest(we["comment_count"]).pvalue
    p_vartest=sp.fligner(wd["comment_count"], we["comment_count"]).pvalue
    
    # TODO: Get the p-value for the t-test

    # TODO: Get the p-value for the normality test on the weekday and weekend data separately
    # That is, are both distributions normal?

    # TODO: Get the p-value for the test that checks whether these two distributions have the same variance
    # ---------- DO NOT CHANGE THE FUNCTION BELOW THIS LINE ---------- #

    if verbose:
        print(f"p_value:\t{p_ttest.round(5)}")
        print(f"WD normality:\t{p_wd_normal.round(5)}")
        print(f"WE normality:\t{p_we_normal.round(5)}")
        print(f"Variance test:\t{p_vartest.round(5)}")

    return p_ttest, p_wd_normal, p_we_normal, p_vartest

In [46]:
tests(wd,we,True)

p_value:	0.0
WD normality:	0.0
WE normality:	0.00152
Variance test:	0.06605


(np.float64(1.3005502847208094e-58),
 np.float64(1.0091137251707994e-07),
 np.float64(0.0015209196859635404),
 np.float64(0.06604578026604031))

In [50]:
raw_df["date"][0].isocalendar()

datetime.IsoCalendarDate(year=2012, week=8, weekday=1)

In [76]:
# TODO - Complete this mewthod
def central_limit_theorem(df: pd.DataFrame) -> pd.DataFrame:
    """
    Combine all weekdays and weekends for each year/week pair and take the average of their
    (untransformed) counts.

    Tip: You can get a "year" and a "week number" from the first two values ​​returned
    by date.isocalendar(). This year and week number will give you an identifier for the (year, week) pair.
    Use Pandas to group by this value and aggregate by taking the average.

    Note: The year returned by isocalendar is not always the same as the year of the date (around New
    Year's Day). Use the year from the isocalendar, which is correct in this case. This is different from the
    year you used to filter events; do not perform any additional filtering!

    Arguments:
    df(pd.DataFrame): Cleaned dataframe containing (at least) the columns: 'date', 'comment_count', 'is_weekend'

    Returns:
    pd.DataFrame: Must have (at least) the columns: 'comment_count', 'is_weekend'
     """
    df = df.copy()

    # TODO: Combine all weekdays and weekends of each year/week pair and take the average of their (untransformed) counts.
    clt: pd.DataFrame = None 

    df['year'] = df['date'].dt.isocalendar().year
    df['week'] = df['date'].dt.isocalendar().week

    clt = df.groupby(['year', 'week','is_weekend'],as_index=False)['comment_count'].mean()

    return clt

In [80]:
other

,date,subreddit,comment_count,weekend
66,2013-03-14,canada,1657,False
69,2013-07-08,canada,1369,False
97,2012-07-04,canada,1343,False
115,2013-03-11,canada,1619,False
165,2013-09-11,canada,1909,False
...,...,...,...,...
15389,2013-01-01,canada,2113,False
15413,2013-07-27,canada,1070,True
15430,2012-10-19,canada,1486,False
15456,2012-01-15,canada,1256,True


In [84]:
clt=central_limit_theorem(other)

In [85]:
clt

,year,week,is_weekend,comment_count
0,2011,52,True,995.0
1,2012,1,False,1561.6
2,2012,1,True,1163.0
3,2012,2,False,2062.8
4,2012,2,True,1372.0
...,...,...,...,...
205,2013,51,False,1964.0
206,2013,51,True,1173.5
207,2013,52,False,1354.4
208,2013,52,True,1117.5


In [68]:
print("Max:", other['comment_count'].max())
print("Min:", other['comment_count'].min())

Max: 3210
Min: 619


In [69]:
print("Max:", clt['comment_count'].max())
print("Min:", clt['comment_count'].min())

Max: 2283.714285714286
Min: 995.0


In [74]:
# TODO - Complete this method
def mann_whitney_u_test(wd: pd.DataFrame, we: pd.DataFrame) -> float:
    """
    Run the Mann-Whitney U test between the weekday and weekend data.

    The Mann-Whitney U test is a nonparametric test that can be used to determine whether samples from one group are larger or smaller than observations from another group. It assumes that both groups have:
    - Independent observations
    - Ordinal values ​​(can be sorted)

    Recall that the alternative hypothesis for a two-sided test states:
    Suppose F(u) and G(u) are the cumulative distribution functions of the distributions
    that yielded x and y, respectively. Then the alternative hypothesis is that the distributions
    are not equal, i.e., F(u) ≠ G(u) for at least one u.

    Arguments:
    wd (pd.DataFrame): Weekday data
    we (pd.DataFrame): Weekend data

    Returns:
    float: p-value of the Mann-Whitney-U test
     """
    # TODO

    p_utest: float = None

    
    p_utest=sp.mannwhitneyu(wd["comment_count"], we["comment_count"]).pvalue
    
    return p_utest

In [75]:
mann_whitney_u_test(wd,we)

np.float64(8.624453234734304e-53)

In [86]:
wd["comment_count"].mean()

np.float64(1823.5785440613026)

In [87]:
we["comment_count"].mean()

np.float64(1269.5071770334928)